We initialize Python imports and opens a DuckDB connection that every later cell reuses

In [ ]:
import duckdb
import pandas as pd

from pathlib import Path

import numpy as np

cwd = Path.cwd()

root = None
for p in [cwd] + list(cwd.parents):
    if (p / "db").exists():
        root = p
        break

if root is None:
    raise FileNotFoundError("Could not find a db folder above the current working directory")

DB_PATH = root / "db" / "nflpa.duckdb"
print("Using DB_PATH", DB_PATH)

con = duckdb.connect(str(DB_PATH))

con.execute("PRAGMA threads=4")
con.execute("PRAGMA memory_limit='4GB'")

print(con.execute("SELECT COUNT(*) AS rows FROM team_week_panel").df())

We confirm that the 'team_week_panel' table, which houses our rows keyed by season, week, and team ID, is physically present in the workspace, sets the team and abbreviation columns to match the configuration used for gathering raw data and building the panel, and also loads the necessary utility functions for the PCA preparation

In [ ]:
tables = set(con.execute("SHOW TABLES").df()["name"].tolist())

if "team_week_panel" not in tables:
    raise RuntimeError("team_week_panel missing, run notebooks 01 through 07 first")

panel_cols = con.execute("PRAGMA table_info('team_week_panel')").df()
panel_cols_list = panel_cols["name"].tolist()
panel_cols_set = set(panel_cols_list)

TEAM_COL = "team_id" if "team_id" in panel_cols_set else "team"
TEAM_ABBR_COL = "team" if "team" in panel_cols_set else TEAM_COL

print("Using TEAM_COL", TEAM_COL)
print("Using TEAM_ABBR_COL", TEAM_ABBR_COL)

def _existing_cols(table_name):
    return set(con.execute(f"PRAGMA table_info('{table_name}')").df()["name"].tolist())

def _star_excluding(table_name, alias, cols_to_maybe_exclude):
    existing = _existing_cols(table_name)
    keep = [c for c in cols_to_maybe_exclude if c in existing]
    if keep:
        return f"{alias}.* EXCLUDE ({', '.join(keep)})"
    return f"{alias}.*"

print("team" in panel_cols_set, "team_id" in panel_cols_set)